In [7]:
import os
import numpy as np
import tensorflow as tf


import utils
import word2vec_utils

In [8]:
tf.reset_default_graph()
VOCAB_SIZE = 50000
BATCH_SIZE = 128
#dimension of the word embedding vectors
EMBED_SIZE = 128
#the context window
SKIP_WINDOW = 2
#number of negative examples to sample
NUM_SAMPLED = 64
LEARNING_RATE = 1.0
NUM_TRAIN_STEPS = 100000
VISUAL_FLD = 'visualization'
SKIP_STEP = 5000

DOWNLOAD_URL = 'http://mattmahoney.net/dc/text8.zip'
EXPECTED_BYTES = 31344016
#number of tokens to visualize
NUM_VISUALIZE = 3000

In [9]:
def word2vec(dataset):
    #Step 1:get input, output from the dataset
    with tf.name_scope('data'):
        iterator = dataset.make_initializable_iterator()
        center_words, target_words = iterator.get_next()
    #Step 2+3:define weights and embedding lookup.
    with tf.name_scope('embed'):
        embed_matrix = tf.get_variable('embed_matrix',shape=[VOCAB_SIZE,EMBED_SIZE],
                                      initializer = tf.random_uniform_initializer())
        embed = tf.nn.embedding_lookup(embed_matrix,center_words,name='embedding')
    #Step 4: construct variables for NCE loss and define loss function
    with tf.name_scope('loss'):
        nce_weight = tf.get_variable('nce_weight',shape=[VOCAB_SIZE,EMBED_SIZE],
                                     initializer=tf.truncated_normal_initializer(stddev=1.0/(EMBED_SIZE**0.5)))
        nce_bias = tf.get_variable('nce_bias',initializer=tf.zeros([VOCAB_SIZE]))
        #define loss function to be NCE loss function
        loss = tf.reduce_mean(tf.nn.nce_loss(weights = nce_weight,
                                            biases = nce_bias,
                                            labels = target_words,
                                            inputs = embed,
                                            num_sampled = NUM_SAMPLED,
                                            num_classes = VOCAB_SIZE),name='loss')
    #Step 5:define optimizer
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)
        utils.safe_mkdir('checkpoints')
        with tf.Session() as sess:
            sess.run(iterator.initializer)
            sess.run(tf.global_variables_initializer())
            total_loss = 0.0
            writer = tf.summary.FileWriter('graphs/word2vec_simple',sess.graph)
            for index in range(NUM_TRAIN_STEPS):
                try:
                    loss_batch, _ = sess.run([loss,optimizer])
                    total_loss+=loss_batch
                    if (index + 1) % SKIP_STEP == 0:
                        print('Average loss at step {}:{:5.1f}'.format(index,total_loss/SKIP_STEP))
                        total_loss = 0.0
                except tf.errors.OutOfRangeError:
                    sess.run(iterator.initializer)
                writer.close()
    

In [12]:
def gen():
    yield from word2vec_utils.batch_gen(DOWNLOAD_URL, EXPECTED_BYTES, VOCAB_SIZE, 
                                        BATCH_SIZE, SKIP_WINDOW, VISUAL_FLD)

def main():
    dataset = tf.data.Dataset.from_generator(gen, 
                                (tf.int32, tf.int32), 
                                (tf.TensorShape([BATCH_SIZE]), tf.TensorShape([BATCH_SIZE, 1])))
    print(dataset)
    word2vec(dataset)


In [13]:
main()

<FlatMapDataset shapes: ((128,), (128, 1)), types: (tf.int32, tf.int32)>


ValueError: Variable embed_matrix already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "D:\Anaconda2\envs\py35\lib\site-packages\tensorflow\python\framework\ops.py", line 1470, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "D:\Anaconda2\envs\py35\lib\site-packages\tensorflow\python\framework\ops.py", line 2956, in create_op
    op_def=op_def)
  File "D:\Anaconda2\envs\py35\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)
